In [1]:
!pip install -qU langchain langchain-community gradio_client python-dotenv langsmith transformers torch tensorflow tf_keras scikit-learn openai

In [2]:
import os
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from gradio_client import Client, handle_file
from langsmith import traceable
from dotenv import load_dotenv
from typing import Union, List, Dict, Any
import json
import mimetypes
import logging
from transformers import pipeline

In [3]:
logging.basicConfig(level=logging.INFO)

In [4]:
from google.colab import userdata
API_KEY = userdata.get('LLAMA_API_KEY')

In [5]:
model_id = "distilbert-base-uncased-finetuned-sst-2-english"
classifier = pipeline("text-classification", model=model_id, top_k=None)

Device set to use cpu


In [8]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name="meta-llama/llama-4-maverick:free",
    base_url="https://openrouter.ai/api/v1",
    api_key=API_KEY,
    temperature=0.3,
    max_tokens=1024,
)

In [9]:
def create_client(client_id: str) -> Client:
    return Client(client_id)

In [10]:
@traceable
def run_text_model(text: str, client_id: str) -> str:
    try:
        client = create_client(client_id)
        label, score = client.predict(text, api_name="/predict")
        return f"Prediction: {label} with a score of {score}"
    except Exception as e:
        logging.error(f"Error in text model {client_id}: {e}")
        return f"Error: {str(e)}"

In [11]:
@traceable
def run_hinglish_hate_model(text: str) -> str:
    return run_text_model(text, "dj-dawgs-ipd/IPD-Hinglish-Text-Model")

In [12]:
@traceable
def run_english_hate_model(text: str) -> str:
    return run_text_model(text, "dj-dawgs-ipd/IPD-Text-English-Finetune")

In [13]:
@traceable
def run_image_hate_model(image_path: str) -> str:
    try:
        client = Client("dj-dawgs-ipd/IPD_IMAGE_PIPELINE")
        result = client.predict(handle_file(image_path), api_name="/predict")
        return f"Image Analysis Result: {result}"
    except Exception as e:
        logging.error(f"Error in image model: {e}")
        return f"Error calling Image model: {str(e)}"

In [14]:
@traceable
def run_audio_hate_model(audio_path: str) -> str:
    try:
        client = Client("dj-dawgs-ipd/IPD-Audio-Pipeline")
        result = client.predict(handle_file(audio_path), api_name="/predict")
        return f"Audio Analysis Result: {result}"
    except Exception as e:
        logging.error(f"Error in audio model: {e}")
        return f"Error calling Audio model: {str(e)}"

In [15]:
@traceable
def run_sentiment_analysis(text: str) -> str:
    try:
        prediction = classifier(text)
        return prediction
        # client = Client("https://ck42-sentiment-model-comparison.hf.space/--replicas/ftdum/")
        # sentiment_data = client.predict(text, api_name="/predict_2")
        # if isinstance(sentiment_data, list) and all(isinstance(item, (list, tuple)) for item in sentiment_data):
        #     formatted_sentiments = ", ".join(
        #         [f"{sentiment} ({score:.2f})" for sentiment, score in sentiment_data]
        #     )
        # else:
        #     formatted_sentiments = str(sentiment_data)
        # return f"Sentiment Analysis: {formatted_sentiments}"
    except Exception as e:
        logging.error(f"Error in sentiment analysis: {e}")
        return f"Error: {str(e)}"

In [16]:
hinglish_tool = Tool.from_function(
    func=run_hinglish_hate_model,
    name="HinglishHateChecker",
    description="Detects hate speech in Hinglish text."
)

In [21]:
english_tool = Tool.from_function(
    func=run_english_hate_model,
    name="EnglishHateChecker",
    description="Detects hate speech in English text."
)

In [17]:
image_tool = Tool.from_function(
    func=run_image_hate_model,
    name="ImageHateChecker",
    description="Detects hate symbols or hateful content in images."
)

In [18]:
audio_tool = Tool.from_function(
    func=run_audio_hate_model,
    name="AudioHateChecker",
    description="Detects hate speech in audio files."
)

In [19]:
sentiment_tool = Tool.from_function(
    func=run_sentiment_analysis,
    name="SentimentAnalyzer",
    description="Analyzes the sentiment of text to assess emotional tone."
)

In [22]:
tools = [hinglish_tool, english_tool, image_tool, audio_tool, sentiment_tool]

In [23]:
def detect_modality(input_data: Union[str, os.PathLike]) -> str:
    if isinstance(input_data, str):
        if os.path.exists(input_data):
            mime_type, _ = mimetypes.guess_type(input_data)
            if mime_type:
                if mime_type.startswith("image"):
                    return "image"
                elif mime_type.startswith("audio"):
                    return "audio"
            return "unknown"
        else:
            return "text"
    return "unknown"

In [24]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True
)

<ipython-input-24-e0edd1dc2c28>:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [25]:
@traceable
def pipeline(input_data: Union[str, os.PathLike]):
    modality = detect_modality(input_data)
    system_instruction = f"""
You are an intelligent AI agent specialized in detecting hate speech. Your objective is to analyze the given input and decide if it constitutes hate speech.

Modality Detected: {modality}

Use the following tools as applicable:
- For text inputs (both English and Hinglish): use the EnglishHateChecker and HinglishHateChecker to detect hate content, and use the SentimentAnalyzer to analyze sentiment.
- For image files: use the ImageHateChecker.
- For audio files: use the AudioHateChecker.

Instructions:
1. **Input Analysis:** First, determine the modality of the input. If it is text, detect the language (English or Hinglish). Document your reasoning as a chain-of-thought.
2. **Tool Invocation:** Based on your analysis, invoke the appropriate tool(s). DO NOT REPEAT USING THE TOOLS IF THEY ARE NOT WORKING. SKIP OVER THEM AND USE YOUR OWN REASONING.
4. **Final Decision:** Using the consolidated outputs and your step-by-step reasoning, decide if the input is hate speech. Your final output must state:
   - "Hate Speech: Yes" or "Hate Speech: No"
   - Followed by "— Reason:" and a comprehensive explanation that integrates all tool outputs and your reasoning.

Final Output Format:
"Hate Speech: Yes/No — Reason: <your detailed explanation with all tool outputs and reasoning>"

IMPORTANT: DO NOT REPEAT USING THE TOOLS IF THEY ARE NOT WORKING. SKIP OVER THEM AND USE YOUR OWN REASONING.
SOMETIMES THE TOOL MAY GIVE WRONG OUTPUT, SO TAKE INTO CONSIDERATION YOUR OWN REASONING AND DO NOT JUSTIFY THE OUTPUT BASED ON THE TOOL OUTPUT.

Now, process the following input:
Input: {input_data}
"""
    prompt = f"{system_instruction}\n\nInput: {input_data}"
    return agent.invoke(prompt)

In [26]:
# print(pipeline("chutiyo logo ko mumbai me lena hi nhi chahiye"))
print(pipeline("swastika.png"))
# print(pipeline("hate_audio.wav"))



> Entering new AgentExecutor chain...
Thought: The input is an image file named "swastika.png". The modality detected is an image. I need to analyze this image to determine if it contains hate symbols or hateful content.

Action: ImageHateChecker
Action Input: swastika.pngLoaded as API: https://dj-dawgs-ipd-ipd-image-pipeline.hf.space ✔

Observation: Image Analysis Result: {'prediction': 'not_hate', 'language': None, 'label': 'No hate found, yay!', 'confidence': None, 'hate_text': None}
Thought:Thought: The ImageHateChecker tool has analyzed the image "swastika.png" and returned a result indicating that it does not contain hate content, with a label stating "No hate found, yay!". However, the swastika is a widely recognized hate symbol associated with Nazi ideology and white supremacy. I need to consider the context and the tool's confidence in its prediction.

Action: None (since the tool has already been invoked)

Thought: Although the ImageHateChecker tool returned a result indica

In [27]:
@traceable
def evaluate_pipeline(test_dataset: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    results = []
    for example in test_dataset:
        inp = example.get("input")
        expected = example.get("expected", "N/A")
        output = pipeline(inp)
        result = {
            "input": inp,
            "expected": expected,
            "output": output,
            "modality": detect_modality(inp)
        }
        results.append(result)
    return results

In [41]:
import psutil
import time
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

def extended_evaluate_pipeline(test_dataset: List[Dict[str, Any]], compute_metrics: bool = True):
    results = []
    predictions = []
    ground_truths = []

    for example in test_dataset:
        inp = example.get("input")
        expected = example.get("expected")  # e.g., "Yes" or "No"

        process = psutil.Process(os.getpid())
        mem_before = process.memory_info().rss

        start_time = time.perf_counter()
        output = pipeline(inp)
        end_time = time.perf_counter()
        processing_time = end_time - start_time

        mem_after = process.memory_info().rss
        mem_used = mem_after - mem_before

        # Assume output follows: "Hate Speech: Yes — Reason: <...>"
        predicted = "Unknown"
        if "Hate Speech:" in output:
            try:
                predicted = output.split("Hate Speech:")[1].split("—")[0].strip()
            except Exception:
                predicted = output.split("Hate Speech:")[1].strip()

        result = {
            "input": inp,
            "expected": expected,
            "output": output,
            "predicted": predicted,
            "processing_time": processing_time,
            "memory_used_bytes": mem_used,
            "modality": detect_modality(inp)
        }
        results.append(result)

        if expected is not None:
            ground_truths.append(expected)
            predictions.append(predicted)

    return results

In [29]:
test_dataset = [
    {"input": "chutiyo logo ko mumbai me lena hi nhi chahiye", "expected": "Yes"},
    {"input": "I love the diverse culture of this city", "expected": "No"},
    {"input": "swastika.png", "expected": "Yes"},
    {"input": "hate_audio.wav", "expected": "Yes"},
]

In [30]:
eval_results = evaluate_pipeline(test_dataset)



> Entering new AgentExecutor chain...
The input is given in Hinglish (a mix of Hindi and English). The first step is to analyze this text for hate speech using the appropriate tool.

Action: HinglishHateChecker
Action Input: chutiyo logo ko mumbai me lena hi nhi chahiyeLoaded as API: https://dj-dawgs-ipd-ipd-hinglish-text-model.hf.space ✔

Observation: Prediction: OAG with a score of 0.9616120457649231
Thought:The HinglishHateChecker tool has detected hate speech in the given text with a high score, indicating a strong likelihood that the text contains hate speech. To further understand the context and sentiment, I will analyze the sentiment of the text.

Action: SentimentAnalyzer
Action Input: chutiyo logo ko mumbai me lena hi nhi chahiye
Observation: [[{'label': 'POSITIVE', 'score': 0.6149528622627258}, {'label': 'NEGATIVE', 'score': 0.3850470781326294}]]
Thought:The sentiment analysis indicates a slightly positive sentiment, which seems counterintuitive given the context. However,

In [31]:
print("\nEvaluation Results:")
for res in eval_results:
    print(json.dumps(res, indent=2))


Evaluation Results:
{
  "input": "chutiyo logo ko mumbai me lena hi nhi chahiye",
  "expected": "Yes",
  "output": {
    "input": "\nYou are an intelligent AI agent specialized in detecting hate speech. Your objective is to analyze the given input and decide if it constitutes hate speech.\n\nModality Detected: text\n\nUse the following tools as applicable:\n- For text inputs (both English and Hinglish): use the EnglishHateChecker and HinglishHateChecker to detect hate content, and use the SentimentAnalyzer to analyze sentiment.\n- For image files: use the ImageHateChecker.\n- For audio files: use the AudioHateChecker.\n\nInstructions:\n1. **Input Analysis:** First, determine the modality of the input. If it is text, detect the language (English or Hinglish). Document your reasoning as a chain-of-thought.\n2. **Tool Invocation:** Based on your analysis, invoke the appropriate tool(s). DO NOT REPEAT USING THE TOOLS IF THEY ARE NOT WORKING. SKIP OVER THEM AND USE YOUR OWN REASONING. \n4.

In [38]:
eval_results = extended_evaluate_pipeline(test_dataset)



> Entering new AgentExecutor chain...
The input is text and appears to be in Hinglish. The first step is to analyze this text for hate speech using the appropriate tool.

Action: HinglishHateChecker
Action Input: chutiyo logo ko mumbai me lena hi nhi chahiyeLoaded as API: https://dj-dawgs-ipd-ipd-hinglish-text-model.hf.space ✔

Observation: Prediction: OAG with a score of 0.9616120457649231
Thought:The HinglishHateChecker tool has returned a prediction of "OAG" with a high score, indicating that the text is likely to be hate speech or offensive. However, to be thorough, I should also analyze the sentiment of the text to understand its emotional tone.

Action: SentimentAnalyzer
Action Input: chutiyo logo ko mumbai me lena hi nhi chahiye
Observation: [[{'label': 'POSITIVE', 'score': 0.6149528622627258}, {'label': 'NEGATIVE', 'score': 0.3850470781326294}]]
Thought:The SentimentAnalyzer tool has returned a mixed sentiment, but leaning towards positive, which seems counterintuitive given 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1618: UserWarning: Note that pos_label (set to 'Yes') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1618: UserWarning: Note that pos_label (set to 'Yes') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 i

In [39]:
print("\nEvaluation Results:")
for res in eval_results:
    print(json.dumps(res, indent=2))


Evaluation Results:
{
  "input": "chutiyo logo ko mumbai me lena hi nhi chahiye",
  "expected": "Yes",
  "output": {
    "input": "\nYou are an intelligent AI agent specialized in detecting hate speech. Your objective is to analyze the given input and decide if it constitutes hate speech.\n\nModality Detected: text\n\nUse the following tools as applicable:\n- For text inputs (both English and Hinglish): use the EnglishHateChecker and HinglishHateChecker to detect hate content, and use the SentimentAnalyzer to analyze sentiment.\n- For image files: use the ImageHateChecker.\n- For audio files: use the AudioHateChecker.\n\nInstructions:\n1. **Input Analysis:** First, determine the modality of the input. If it is text, detect the language (English or Hinglish). Document your reasoning as a chain-of-thought.\n2. **Tool Invocation:** Based on your analysis, invoke the appropriate tool(s). DO NOT REPEAT USING THE TOOLS IF THEY ARE NOT WORKING. SKIP OVER THEM AND USE YOUR OWN REASONING. \n4.